# Graphcut

In [1]:
!pip install pygco imcut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pygco: filename=pygco-0.0.16-cp38-cp38-linux_x86_64.whl size=373453 sha256=16f5e39c6f04dda0800fbf62546cf615a38566f67ab311f71cbb37f881bb58cc
  Stored in directory: /root/.cache/pip/wheels/c7/a2/c6/edae53a0994ed7913c901663dfc7cba042067857a7cc05ed42
  Created wheel for imcut: filename=imcut-1.9.4-py3-none-any.whl size=53541 sha256=8dc9f086f1aa12da3af4343597667dbbb5db058e0526b0fabc24e2386af38c84
  Stored in directory: /root/.cache/pip/wheels/fa/a1/5b/fac311f8c8743806ba97f2c2406469b75a7557d17c937c6fcf
Successfully built pygco imcut


In [2]:
# import core libraries
import IPython
import numpy as np
import base64
import re
import imcut.pycut
import matplotlib.pyplot as plt
import cv2

from PIL import Image
from io import BytesIO
from google.colab import output

In [3]:
def base64ToImage(base64Img:str, cv2flag = cv2.IMREAD_UNCHANGED) -> np.array:
    """
        decodes the given based-64 image

        Parameters
            base64Img (str) image encoded as base64
            cv2flag (int) flags used for image file reading and writing

        Returns
            np.array image
    """
    encodedData = re.sub('^data:image/.+;base64,', '', base64Img)
    decoded_data = base64.b64decode(encodedData)
    np_data = np.frombuffer(decoded_data,np.uint8)
    return cv2.imdecode(np_data, cv2flag)

In [4]:
def toBase64(arr: np.array) -> str:
    """
        Encodes the given image to base64 

        Parameters
            arr (np.array) encodes the given image to base64

        Returns
            str image encoded as base64
    """
    generatedImage = Image.fromarray(arr)
    buffered = BytesIO()
    generatedImage.save(buffered, format="PNG")
    return 'data:image/png;base64,' + base64.b64encode(buffered.getvalue()).decode('utf-8')

In [5]:
def filter(image : np.array, mask: np.array) -> np.array:
    """ 
        Generates an image based on the given image and mask. 
        The mask is a np.array containg 0s and 1s.
        Only pixels with value 0s get kept

        Parameters
            image (np.array) source image
            mask (np.array) pixels that will be kept

        Returns
            np.array filtered image
    """
    segmentedImage = np.zeros(image.shape, dtype="uint8")
    for x in range(mask.shape[0]):
        for y in range(mask.shape[1]):
            isForeground = mask[x, y] == 0;
            if (isForeground):
                segmentedImage[x, y] = image[x, y]
    return segmentedImage

In [6]:
def encodeScribbles(scribblesImage:np.array) -> np.array:
    """
        Creates a np.array based on the given image.
        Blue pixels get encoded as 1 (1 means foreground / object)
        Red pixels get encoded as 2 (2 means background)

        Parameters
            scribblesImage (np.array) image containg green and blue pixels only
        
        Returns
            np.array containg 1s and 2s
    """

    seeds = np.zeros((scribblesImage.shape[0], scribblesImage.shape[1]), dtype="uint8")
    for x in range(scribblesImage.shape[0]):
        for y in range(scribblesImage.shape[1]):
            pixel = np.array2string(scribblesImage[x, y])
            if (pixel == '[  0   0 255]'):      # FOREGROUND
                seeds[x, y] = 1
            elif (pixel == '[  0 255   0]'):    # BACKGROUND
                seeds[x, y] = 2
    return seeds

In [7]:
def graphCut(image : np.array, scribbles: np.array) -> np.array:
    """
        Identifies pixels that are foreground and background based on the given image
        and the scribbles.

        image and scribbles are expected to be the same size

        Parameters
            image (np.array) source image
            scribbles (np.array) image containing seeds (pixels blue and green)

        Returns
            np.array containg 0s and 1s
                where 
                    0 means a pixel that is part of foreground / object
                    1 means a pixel that is part of background
    """
    newShape = (image.shape[0], image.shape[1], -1)
    seeds = encodeScribbles(scribbles)

    image3d = np.reshape(image, newShape)
    seeds3d = np.reshape(seeds, (image.shape[0], image.shape[1], -1))

    # init instance of ImageGraphCut
    gc = imcut.pycut.ImageGraphCut(image3d)
    # setting seeds
    gc.set_seeds(seeds3d)
    # executing graph cut
    gc.run()

    return gc.segmentation.squeeze()

In [11]:
def segment(original : str, scribbled: str) -> str:
    """
        Peforms Graph-cut segmentation

        Parameters
            original (str) source image encoded as base64
            scribbled (str) seeds image encoded as base64 (blue and green pixels only)

        Returns
            str segmented image as base64
    """

    image = base64ToImage(original)
    grayScaledImage = base64ToImage(original, cv2.IMREAD_GRAYSCALE)
    scribbledImage = base64ToImage(scribbled, cv2.IMREAD_COLOR)
    encodedSegmentation = graphCut(grayScaledImage, scribbledImage)
    segmentedImage = filter(image, encodedSegmentation)
    return toBase64(segmentedImage)

Para la implementación de este demo se uso la libreria imcut. Este es un tool de segmentación basado en el algoritmo de graph cut.

Jirik, M., Lukes, V., Svobodova, M., & Zelezny, M. (2013). imcut | 3D graph cut segmentation. GitHub Pages. Retrieved March 13, 2023, from http://mjirik.github.io/imcut/

In [12]:
# register segment method in order to make it available for the frontend
output.register_callback('notebook.segment', segment)

In [20]:
# renders the frontend
display(IPython.display.HTML('''
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Vite App</title>
    <style>
      *,:before,:after{box-sizing:border-box;border-width:0;border-style:solid;border-color:#e5e7eb}:before,:after{--tw-content: ""}html{line-height:1.5;-webkit-text-size-adjust:100%;-moz-tab-size:4;-o-tab-size:4;tab-size:4;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,"Apple Color Emoji","Segoe UI Emoji",Segoe UI Symbol,"Noto Color Emoji";font-feature-settings:normal}body{margin:0;line-height:inherit}hr{height:0;color:inherit;border-top-width:1px}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,samp,pre{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{text-indent:0;border-color:inherit;border-collapse:collapse}button,input,optgroup,select,textarea{font-family:inherit;font-size:100%;font-weight:inherit;line-height:inherit;color:inherit;margin:0;padding:0}button,select{text-transform:none}button,[type=button],[type=reset],[type=submit]{-webkit-appearance:button;background-color:transparent;background-image:none}:-moz-focusring{outline:auto}:-moz-ui-invalid{box-shadow:none}progress{vertical-align:baseline}::-webkit-inner-spin-button,::-webkit-outer-spin-button{height:auto}[type=search]{-webkit-appearance:textfield;outline-offset:-2px}::-webkit-search-decoration{-webkit-appearance:none}::-webkit-file-upload-button{-webkit-appearance:button;font:inherit}summary{display:list-item}blockquote,dl,dd,h1,h2,h3,h4,h5,h6,hr,figure,p,pre{margin:0}fieldset{margin:0;padding:0}legend{padding:0}ol,ul,menu{list-style:none;margin:0;padding:0}textarea{resize:vertical}input::-moz-placeholder,textarea::-moz-placeholder{opacity:1;color:#9ca3af}input::placeholder,textarea::placeholder{opacity:1;color:#9ca3af}button,[role=button]{cursor:pointer}:disabled{cursor:default}img,svg,video,canvas,audio,iframe,embed,object{display:block;vertical-align:middle}img,video{max-width:100%;height:auto}[hidden]{display:none}[type=text],[type=email],[type=url],[type=password],[type=number],[type=date],[type=datetime-local],[type=month],[type=search],[type=tel],[type=time],[type=week],[multiple],textarea,select{-webkit-appearance:none;-moz-appearance:none;appearance:none;background-color:#fff;border-color:#6b7280;border-width:1px;border-radius:0;padding:.5rem .75rem;font-size:1rem;line-height:1.5rem;--tw-shadow: 0 0 #0000}[type=text]:focus,[type=email]:focus,[type=url]:focus,[type=password]:focus,[type=number]:focus,[type=date]:focus,[type=datetime-local]:focus,[type=month]:focus,[type=search]:focus,[type=tel]:focus,[type=time]:focus,[type=week]:focus,[multiple]:focus,textarea:focus,select:focus{outline:2px solid transparent;outline-offset:2px;--tw-ring-inset: var(--tw-empty, );--tw-ring-offset-width: 0px;--tw-ring-offset-color: #fff;--tw-ring-color: #2563eb;--tw-ring-offset-shadow: var(--tw-ring-inset) 0 0 0 var(--tw-ring-offset-width) var(--tw-ring-offset-color);--tw-ring-shadow: var(--tw-ring-inset) 0 0 0 calc(1px + var(--tw-ring-offset-width)) var(--tw-ring-color);box-shadow:var(--tw-ring-offset-shadow),var(--tw-ring-shadow),var(--tw-shadow);border-color:#2563eb}input::-moz-placeholder,textarea::-moz-placeholder{color:#6b7280;opacity:1}input::placeholder,textarea::placeholder{color:#6b7280;opacity:1}::-webkit-datetime-edit-fields-wrapper{padding:0}::-webkit-date-and-time-value{min-height:1.5em}::-webkit-datetime-edit,::-webkit-datetime-edit-year-field,::-webkit-datetime-edit-month-field,::-webkit-datetime-edit-day-field,::-webkit-datetime-edit-hour-field,::-webkit-datetime-edit-minute-field,::-webkit-datetime-edit-second-field,::-webkit-datetime-edit-millisecond-field,::-webkit-datetime-edit-meridiem-field{padding-top:0;padding-bottom:0}select{background-image:url("data:image/svg+xml,%3csvg xmlns='http://www.w3.org/2000/svg' fill='none' viewBox='0 0 20 20'%3e%3cpath stroke='%236b7280' stroke-linecap='round' stroke-linejoin='round' stroke-width='1.5' d='M6 8l4 4 4-4'/%3e%3c/svg%3e");background-position:right .5rem center;background-repeat:no-repeat;background-size:1.5em 1.5em;padding-right:2.5rem;-webkit-print-color-adjust:exact;print-color-adjust:exact}[multiple]{background-image:initial;background-position:initial;background-repeat:unset;background-size:initial;padding-right:.75rem;-webkit-print-color-adjust:unset;print-color-adjust:unset}[type=checkbox],[type=radio]{-webkit-appearance:none;-moz-appearance:none;appearance:none;padding:0;-webkit-print-color-adjust:exact;print-color-adjust:exact;display:inline-block;vertical-align:middle;background-origin:border-box;-webkit-user-select:none;-moz-user-select:none;user-select:none;flex-shrink:0;height:1rem;width:1rem;color:#2563eb;background-color:#fff;border-color:#6b7280;border-width:1px;--tw-shadow: 0 0 #0000}[type=checkbox]{border-radius:0}[type=radio]{border-radius:100%}[type=checkbox]:focus,[type=radio]:focus{outline:2px solid transparent;outline-offset:2px;--tw-ring-inset: var(--tw-empty, );--tw-ring-offset-width: 2px;--tw-ring-offset-color: #fff;--tw-ring-color: #2563eb;--tw-ring-offset-shadow: var(--tw-ring-inset) 0 0 0 var(--tw-ring-offset-width) var(--tw-ring-offset-color);--tw-ring-shadow: var(--tw-ring-inset) 0 0 0 calc(2px + var(--tw-ring-offset-width)) var(--tw-ring-color);box-shadow:var(--tw-ring-offset-shadow),var(--tw-ring-shadow),var(--tw-shadow)}[type=checkbox]:checked,[type=radio]:checked{border-color:transparent;background-color:currentColor;background-size:100% 100%;background-position:center;background-repeat:no-repeat}[type=checkbox]:checked{background-image:url("data:image/svg+xml,%3csvg viewBox='0 0 16 16' fill='white' xmlns='http://www.w3.org/2000/svg'%3e%3cpath d='M12.207 4.793a1 1 0 010 1.414l-5 5a1 1 0 01-1.414 0l-2-2a1 1 0 011.414-1.414L6.5 9.086l4.293-4.293a1 1 0 011.414 0z'/%3e%3c/svg%3e")}[type=radio]:checked{background-image:url("data:image/svg+xml,%3csvg viewBox='0 0 16 16' fill='white' xmlns='http://www.w3.org/2000/svg'%3e%3ccircle cx='8' cy='8' r='3'/%3e%3c/svg%3e")}[type=checkbox]:checked:hover,[type=checkbox]:checked:focus,[type=radio]:checked:hover,[type=radio]:checked:focus{border-color:transparent;background-color:currentColor}[type=checkbox]:indeterminate{background-image:url("data:image/svg+xml,%3csvg xmlns='http://www.w3.org/2000/svg' fill='none' viewBox='0 0 16 16'%3e%3cpath stroke='white' stroke-linecap='round' stroke-linejoin='round' stroke-width='2' d='M4 8h8'/%3e%3c/svg%3e");border-color:transparent;background-color:currentColor;background-size:100% 100%;background-position:center;background-repeat:no-repeat}[type=checkbox]:indeterminate:hover,[type=checkbox]:indeterminate:focus{border-color:transparent;background-color:currentColor}[type=file]{background:unset;border-color:inherit;border-width:0;border-radius:0;padding:0;font-size:unset;line-height:inherit}[type=file]:focus{outline:1px solid ButtonText;outline:1px auto -webkit-focus-ring-color}*,:before,:after{--tw-border-spacing-x: 0;--tw-border-spacing-y: 0;--tw-translate-x: 0;--tw-translate-y: 0;--tw-rotate: 0;--tw-skew-x: 0;--tw-skew-y: 0;--tw-scale-x: 1;--tw-scale-y: 1;--tw-pan-x: ;--tw-pan-y: ;--tw-pinch-zoom: ;--tw-scroll-snap-strictness: proximity;--tw-ordinal: ;--tw-slashed-zero: ;--tw-numeric-figure: ;--tw-numeric-spacing: ;--tw-numeric-fraction: ;--tw-ring-inset: ;--tw-ring-offset-width: 0px;--tw-ring-offset-color: #fff;--tw-ring-color: rgb(59 130 246 / .5);--tw-ring-offset-shadow: 0 0 #0000;--tw-ring-shadow: 0 0 #0000;--tw-shadow: 0 0 #0000;--tw-shadow-colored: 0 0 #0000;--tw-blur: ;--tw-brightness: ;--tw-contrast: ;--tw-grayscale: ;--tw-hue-rotate: ;--tw-invert: ;--tw-saturate: ;--tw-sepia: ;--tw-drop-shadow: ;--tw-backdrop-blur: ;--tw-backdrop-brightness: ;--tw-backdrop-contrast: ;--tw-backdrop-grayscale: ;--tw-backdrop-hue-rotate: ;--tw-backdrop-invert: ;--tw-backdrop-opacity: ;--tw-backdrop-saturate: ;--tw-backdrop-sepia: }::backdrop{--tw-border-spacing-x: 0;--tw-border-spacing-y: 0;--tw-translate-x: 0;--tw-translate-y: 0;--tw-rotate: 0;--tw-skew-x: 0;--tw-skew-y: 0;--tw-scale-x: 1;--tw-scale-y: 1;--tw-pan-x: ;--tw-pan-y: ;--tw-pinch-zoom: ;--tw-scroll-snap-strictness: proximity;--tw-ordinal: ;--tw-slashed-zero: ;--tw-numeric-figure: ;--tw-numeric-spacing: ;--tw-numeric-fraction: ;--tw-ring-inset: ;--tw-ring-offset-width: 0px;--tw-ring-offset-color: #fff;--tw-ring-color: rgb(59 130 246 / .5);--tw-ring-offset-shadow: 0 0 #0000;--tw-ring-shadow: 0 0 #0000;--tw-shadow: 0 0 #0000;--tw-shadow-colored: 0 0 #0000;--tw-blur: ;--tw-brightness: ;--tw-contrast: ;--tw-grayscale: ;--tw-hue-rotate: ;--tw-invert: ;--tw-saturate: ;--tw-sepia: ;--tw-drop-shadow: ;--tw-backdrop-blur: ;--tw-backdrop-brightness: ;--tw-backdrop-contrast: ;--tw-backdrop-grayscale: ;--tw-backdrop-hue-rotate: ;--tw-backdrop-invert: ;--tw-backdrop-opacity: ;--tw-backdrop-saturate: ;--tw-backdrop-sepia: }.sr-only{position:absolute;width:1px;height:1px;padding:0;margin:-1px;overflow:hidden;clip:rect(0,0,0,0);white-space:nowrap;border-width:0}.absolute{position:absolute}.relative{position:relative}.inset-0{top:0px;right:0px;bottom:0px;left:0px}.top-0{top:0px}.col-span-2{grid-column:span 2 / span 2}.mb-4{margin-bottom:1rem}.mr-3{margin-right:.75rem}.mt-0{margin-top:0}.mt-4{margin-top:1rem}.block{display:block}.flex{display:flex}.grid{display:grid}.hidden{display:none}.h-4{height:1rem}.h-6{height:1.5rem}.h-8{height:2rem}.h-full{height:100%}.h-screen{height:100vh}.w-4{width:1rem}.w-6{width:1.5rem}.w-8{width:2rem}.w-96{width:24rem}.w-full{width:100%}.flex-1{flex:1 1 0%}@keyframes spin{to{transform:rotate(360deg)}}.animate-spin{animation:spin 1s linear infinite}.resize{resize:both}.grid-cols-2{grid-template-columns:repeat(2,minmax(0,1fr))}.grid-cols-3{grid-template-columns:repeat(3,minmax(0,1fr))}.items-start{align-items:flex-start}.items-center{align-items:center}.justify-center{justify-content:center}.gap-2{gap:.5rem}.gap-4{gap:1rem}.gap-5{gap:1.25rem}.space-y-2>:not([hidden])~:not([hidden]){--tw-space-y-reverse: 0;margin-top:calc(.5rem * calc(1 - var(--tw-space-y-reverse)));margin-bottom:calc(.5rem * var(--tw-space-y-reverse))}.divide-y>:not([hidden])~:not([hidden]){--tw-divide-y-reverse: 0;border-top-width:calc(1px * calc(1 - var(--tw-divide-y-reverse)));border-bottom-width:calc(1px * var(--tw-divide-y-reverse))}.rounded{border-radius:.25rem}.rounded-md{border-radius:.375rem}.border{border-width:1px}.border-gray-300{--tw-border-opacity: 1;border-color:rgb(209 213 219 / var(--tw-border-opacity))}.bg-blue-50{--tw-bg-opacity: 1;background-color:rgb(239 246 255 / var(--tw-bg-opacity))}.bg-gray-900\/60{background-color:#11182799}.bg-indigo-600{--tw-bg-opacity: 1;background-color:rgb(79 70 229 / var(--tw-bg-opacity))}.bg-slate-50{--tw-bg-opacity: 1;background-color:rgb(248 250 252 / var(--tw-bg-opacity))}.bg-white{--tw-bg-opacity: 1;background-color:rgb(255 255 255 / var(--tw-bg-opacity))}.fill-blue-400{fill:#60a5fa}.fill-slate-600{fill:#475569}.p-2{padding:.5rem}.p-4{padding:1rem}.px-3{padding-left:.75rem;padding-right:.75rem}.px-3\.5{padding-left:.875rem;padding-right:.875rem}.px-4{padding-left:1rem;padding-right:1rem}.py-2{padding-top:.5rem;padding-bottom:.5rem}.py-2\.5{padding-top:.625rem;padding-bottom:.625rem}.py-4{padding-top:1rem;padding-bottom:1rem}.text-center{text-align:center}.text-2xl{font-size:1.5rem;line-height:2rem}.text-lg{font-size:1.125rem;line-height:1.75rem}.text-sm{font-size:.875rem;line-height:1.25rem}.font-bold{font-weight:700}.font-light{font-weight:300}.font-medium{font-weight:500}.font-semibold{font-weight:600}.tracking-wide{letter-spacing:.025em}.text-blue-400{--tw-text-opacity: 1;color:rgb(96 165 250 / var(--tw-text-opacity))}.text-gray-900{--tw-text-opacity: 1;color:rgb(17 24 39 / var(--tw-text-opacity))}.text-indigo-600{--tw-text-opacity: 1;color:rgb(79 70 229 / var(--tw-text-opacity))}.text-slate-500{--tw-text-opacity: 1;color:rgb(100 116 139 / var(--tw-text-opacity))}.text-slate-600{--tw-text-opacity: 1;color:rgb(71 85 105 / var(--tw-text-opacity))}.text-slate-900{--tw-text-opacity: 1;color:rgb(15 23 42 / var(--tw-text-opacity))}.text-white{--tw-text-opacity: 1;color:rgb(255 255 255 / var(--tw-text-opacity))}.opacity-25{opacity:.25}.opacity-75{opacity:.75}.shadow{--tw-shadow: 0 1px 3px 0 rgb(0 0 0 / .1), 0 1px 2px -1px rgb(0 0 0 / .1);--tw-shadow-colored: 0 1px 3px 0 var(--tw-shadow-color), 0 1px 2px -1px var(--tw-shadow-color);box-shadow:var(--tw-ring-offset-shadow, 0 0 #0000),var(--tw-ring-shadow, 0 0 #0000),var(--tw-shadow)}.shadow-sm{--tw-shadow: 0 1px 2px 0 rgb(0 0 0 / .05);--tw-shadow-colored: 0 1px 2px 0 var(--tw-shadow-color);box-shadow:var(--tw-ring-offset-shadow, 0 0 #0000),var(--tw-ring-shadow, 0 0 #0000),var(--tw-shadow)}.hover\:bg-gray-50:hover{--tw-bg-opacity: 1;background-color:rgb(249 250 251 / var(--tw-bg-opacity))}.hover\:bg-indigo-500:hover{--tw-bg-opacity: 1;background-color:rgb(99 102 241 / var(--tw-bg-opacity))}.focus\:ring-indigo-600:focus{--tw-ring-opacity: 1;--tw-ring-color: rgb(79 70 229 / var(--tw-ring-opacity))}.active\:bg-gray-200:active{--tw-bg-opacity: 1;background-color:rgb(229 231 235 / var(--tw-bg-opacity))}.active\:bg-indigo-700:active{--tw-bg-opacity: 1;background-color:rgb(67 56 202 / var(--tw-bg-opacity))}
    </style>
  </head>
  <body>
    <div class="h-screen flex bg-slate-50 relative">
        <aside class="w-96 bg-white shadow">
            <div class="px-4">
                <div class="mt-4">
                    <h1 class="text-2xl font-bold text-slate-900">Graph-Cut</h1>
                </div>
                <div class="divide-y">
                    <section class="py-4">
                        <span class="block text-indigo-600 font-medium">Step 1</span>
                        <div class="flex items-center gap-2 mb-4">
                            <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="w-6 h-6 fill-slate-600" fill="currentColor">
                                <path fill-rule="evenodd" d="M1.5 6a2.25 2.25 0 012.25-2.25h16.5A2.25 2.25 0 0122.5 6v12a2.25 2.25 0 01-2.25 2.25H3.75A2.25 2.25 0 011.5 18V6zM3 16.06V18c0 .414.336.75.75.75h16.5A.75.75 0 0021 18v-1.94l-2.69-2.689a1.5 1.5 0 00-2.12 0l-.88.879.97.97a.75.75 0 11-1.06 1.06l-5.16-5.159a1.5 1.5 0 00-2.12 0L3 16.061zm10.125-7.81a1.125 1.125 0 112.25 0 1.125 1.125 0 01-2.25 0z" clip-rule="evenodd" />
                            </svg>
                            <h2 class="text-lg font-medium text-slate-900">Upload image</h2>
                        </div>
                        <label for="file-upload" class="block w-full text-center rounded-md border border-gray-300 bg-white px-3.5 py-2.5 text-sm font-semibold text-gray-900 shadow-sm hover:bg-gray-50">
                            <span>Select an image</span>
                            <input id="file-upload" name="file-upload" type="file" class="sr-only" accept="image/*">
                        </label>
                    </section>
                    <section class="py-4">
                        <span class="block text-indigo-600 font-medium">Step 2</span>
                        <div class="flex items-center gap-2 mb-4">
                            <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" class="w-6 h-6 fill-slate-600" fill="currentColor">
                                <path fill-rule="evenodd" d="M20.599 1.5c-.376 0-.743.111-1.055.32l-5.08 3.385a18.747 18.747 0 00-3.471 2.987 10.04 10.04 0 014.815 4.815 18.748 18.748 0 002.987-3.472l3.386-5.079A1.902 1.902 0 0020.599 1.5zm-8.3 14.025a18.76 18.76 0 001.896-1.207 8.026 8.026 0 00-4.513-4.513A18.75 18.75 0 008.475 11.7l-.278.5a5.26 5.26 0 013.601 3.602l.502-.278zM6.75 13.5A3.75 3.75 0 003 17.25a1.5 1.5 0 01-1.601 1.497.75.75 0 00-.7 1.123 5.25 5.25 0 009.8-2.62 3.75 3.75 0 00-3.75-3.75z" clip-rule="evenodd" />
                            </svg>
                            <h2 class="text-lg font-medium text-slate-900">Seed image</h2>
                        </div>
                        <div class="space-y-2">
                            <fieldset id="brush-selector" class="space-y-2">
                                <label class="flex items-center gap-2">
                                    <input type="radio" name="brush" value="foreground" class="h-4 w-4 border-gray-300 text-indigo-600 focus:ring-indigo-600"/>
                                    <svg class="h-8 w-8 fill-slate-600" viewBox="0 0 100 100" fill="currentColor" xmlns="http://www.w3.org/2000/svg" >
                                        <rect x="15.5" y="15.5" width="49" height="49" rx="4.5" fill="#F1F5F9" stroke="#6B7280" stroke-dasharray="2 2"/>
                                        <rect x="35.5" y="35.5" width="49" height="49" rx="4.5" />
                                    </svg>
                                    <span class="text-slate-600">Foreground brush</span>
                                </label>
                                <label class="flex items-center gap-2">
                                    <input type="radio" name="brush" value="background" class="h-4 w-4 border-gray-300 text-indigo-600 focus:ring-indigo-600"/>
                                    <svg class="h-8 w-8 fill-slate-600" viewBox="0 0 100 100" fill="currentColor" xmlns="http://www.w3.org/2000/svg">
                                        <rect x="15.5" y="15.5" width="49" height="49" rx="4.5" />
                                        <rect x="35.5" y="35.5" width="49" height="49" rx="4.5" fill="#F1F5F9" stroke="#6B7280" stroke-dasharray="2 2"/>
                                    </svg>
                                    <span class="text-slate-600">Background brush</span>
                                </label>
                            </fieldset>
                            <button id="eraser" type="button" class="w-full rounded-md border border-gray-300 bg-white px-3.5 py-2.5 text-sm font-semibold text-gray-900 shadow-sm hover:bg-gray-50 active:bg-gray-200">
                                Clear Seeds
                            </button>
                        </div>
                    </section>
                    <section class="py-4">
                        <span class="block text-indigo-600 font-medium">Step 3</span>
                        <div class="flex items-center gap-2 mb-4">
                            <svg class="h-8 w-8 fill-slate-600" viewBox="0 0 100 100" fill="currentColor" xmlns="http://www.w3.org/2000/svg">                     
                                <path d="M60 10C40 30 60 70 40 90" stroke="black"/>
                                <line x1="10.5" y1="41" x2="10.5" y2="61" stroke="black"/>
                                <line x1="9.64645" y1="39.6464" x2="24.6464" y2="24.6464" stroke="black"/>
                                <line x1="10.3536" y1="59.6464" x2="25.3536" y2="74.6464" stroke="black"/>
                                <line x1="25.4642" y1="24.8143" x2="35.4642" y2="49.8143" stroke="black"/>
                                <line x1="10.1857" y1="39.5358" x2="35.1857" y2="49.5358" stroke="black"/>
                                <line x1="9.8143" y1="59.5358" x2="34.8143" y2="49.5358" stroke="black"/>
                                <line x1="24.5358" y1="74.8143" x2="34.5358" y2="49.8143" stroke="black"/>
                                <line x1="75.3536" y1="24.6464" x2="90.3536" y2="39.6464" stroke="black"/>
                                <line x1="90.5" y1="40" x2="90.5" y2="60" stroke="black"/>
                                <line x1="90.3536" y1="60.3536" x2="75.3536" y2="75.3536" stroke="black"/>
                                <line x1="75.4642" y1="25.1857" x2="65.4642" y2="50.1857" stroke="black"/>
                                <line x1="74.5358" y1="75.1857" x2="64.5358" y2="50.1857" stroke="black"/>
                                <line x1="90.1857" y1="40.4642" x2="65.1857" y2="50.4642" stroke="black"/>
                                <line x1="89.8143" y1="60.4642" x2="64.8143" y2="50.4642" stroke="black"/>
                                <circle cx="25" cy="25" r="7.5"/>
                                <circle cx="75" cy="25" r="7.5"/>
                                <circle cx="10" cy="40" r="7.5"/>
                                <circle cx="90" cy="40" r="7.5"/>
                                <circle cx="10" cy="60" r="7.5"/>
                                <circle cx="90" cy="60" r="7.5"/>
                                <circle cx="25" cy="75" r="7.5"/>
                                <circle cx="75" cy="75" r="7.5"/>
                                <circle cx="35" cy="50" r="7.5"/>
                                <circle cx="65" cy="50" r="7.5"/>
                            </svg>
                            <h2 class="text-lg font-medium text-slate-900">Graph-cut</h2>
                        </div>
                        <div class="p-2 bg-blue-50 rounded mb-4">
                            <div class="flex gap-2 items-start">
                                <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" fill="currentColor" class="w-8 h-8 fill-blue-400">
                                    <path fill-rule="evenodd" d="M2.25 12c0-5.385 4.365-9.75 9.75-9.75s9.75 4.365 9.75 9.75-4.365 9.75-9.75 9.75S2.25 17.385 2.25 12zm8.706-1.442c1.146-.573 2.437.463 2.126 1.706l-.709 2.836.042-.02a.75.75 0 01.67 1.34l-.04.022c-1.147.573-2.438-.463-2.127-1.706l.71-2.836-.042.02a.75.75 0 11-.671-1.34l.041-.022zM12 9a.75.75 0 100-1.5.75.75 0 000 1.5z" clip-rule="evenodd" />
                                </svg>
                                <p class="text-blue-400">
                                    Make sure image gets seeded before executing <b>Segment</b> in order to get an accurate result
                                </p>
                            </div>
                        </div>
                        <button id="graph-cut" class="w-full rounded-md bg-indigo-600 px-3.5 py-2.5 text-sm font-semibold text-white shadow-sm hover:bg-indigo-500 active:bg-indigo-700">
                            Segment
                        </button>
                    </section>
                </div>
            </div>
        </aside>
        <main class="flex-1">
            <div class="flex items-center justify-center h-full">
                <section id="placeholder">
                    <div>
                        <div class="text-center">
                            <h2 class="text-slate-900 text-2xl font-semibold tracking-wide">No image selected</h2>
                            <p class="text-slate-600 font-light">Get started by uploading an image</p>
                        </div>
                    </div>
                </section>

                <div id="scribble-container" class="relative"></div>

                <div id="result-container" class="hidden p-4 bg-white shadow rounded">
                    <div class="grid grid-cols-2 gap-4 mb-4">
                        <div class="space-y-2">
                            <div class="flex items-center justify-center">
                               <img id="output-original" alt="Original"/>
                            </div>
                            <span class="block text-center text-slate-500 font-semibold">Original</span>
                        </div>
                
                        <div class="space-y-2">
                            <div class="flex items-center justify-center">
                                <img id="output-segmented" alt="Segmented"/>
                             </div>
                            <span class="block text-center text-slate-500 font-semibold">Segmented</span>
                        </div>
                    </div>
                    <div class="">
                        <dl>
                            <div class="py-2 grid grid-cols-3 gap-5 text-sm">
                                <dt class="font-medium text-slate-500">Image size</dt>
                                <dd id="output-dimension" class="text-slate-900 col-span-2 mt-0"></dd>
                            </div>
                            <div class="py-2 grid grid-cols-3 gap-5 text-sm">
                                <dt class="font-medium text-slate-500">Processing time</dt>
                                <dd id="output-processing-time" class="text-slate-900 col-span-2 mt-0"></dd>
                            </div>
                        </dl>
                    </div>
                </div>
            </div>
        </main>
        <div id="spinner" class="hidden absolute inset-0 bg-gray-900/60">
            <div class="h-full flex items-center justify-center">
                <div class="flex items-center">
                    <svg class="animate-spin mr-3 h-8 w-8 text-white" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                        <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                        <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"/>
                    </svg>
                    <span class="font-semibold text-white text-2xl">Processing...</span>
                </div>
            </div>
        </div>
    </div>
    <script>
      (function(){const e=document.createElement("link").relList;if(e&&e.supports&&e.supports("modulepreload"))return;for(const t of document.querySelectorAll('link[rel="modulepreload"]'))s(t);new MutationObserver(t=>{for(const o of t)if(o.type==="childList")for(const c of o.addedNodes)c.tagName==="LINK"&&c.rel==="modulepreload"&&s(c)}).observe(document,{childList:!0,subtree:!0});function n(t){const o={};return t.integrity&&(o.integrity=t.integrity),t.referrerPolicy&&(o.referrerPolicy=t.referrerPolicy),t.crossOrigin==="use-credentials"?o.credentials="include":t.crossOrigin==="anonymous"?o.credentials="omit":o.credentials="same-origin",o}function s(t){if(t.ep)return;t.ep=!0;const o=n(t);fetch(t.href,o)}})();function P(i){const e=new window.Image;return e.src=window.URL.createObjectURL(i),new Promise(n=>{e.onload=()=>{n(e)}})}function I(i){const e=i.getContext("2d"),n={x:0,y:0},s={x:0,y:0};let t=!1;i.addEventListener("mousemove",o,!1),e.lineWidth=5,e.lineJoin="round",e.lineCap="round",e.strokeStyle="blue",i.addEventListener("mousedown",function(){i.addEventListener("mousemove",c,!1)},!1),i.addEventListener("mouseup",function(){i.removeEventListener("mousemove",c,!1)},!1);function o(l){s.x=n.x,s.y=n.y,n.x=l.layerX,n.y=l.layerY}function c(){t&&(e.beginPath(),e.moveTo(s.x,s.y),e.lineTo(n.x,n.y),e.closePath(),e.stroke())}function h(l){t=!0,e.strokeStyle=l}function f(){e.clearRect(0,0,i.width,i.height)}function u(){f(),i.removeEventListener("mousemove",o,!1),i.remove()}return{setStrokeColor:h,clear:f,dispose:u}}const O={foreground:"rgb(255, 0, 0)",background:"rgb(0, 255, 0)"};function R(){let i=null,e=null,n=null,s=null;const t=document.querySelector("#file-upload"),o=document.querySelector("#eraser"),c=document.querySelector("#graph-cut"),h=document.querySelector("#placeholder"),f=document.querySelector("#brush-selector"),u=document.querySelector("#scribble-container"),l=document.querySelector("#result-container"),y=document.querySelector("#spinner");function E(){return t.addEventListener("change",b),o.addEventListener("click",w),c.addEventListener("click",S),f.addEventListener("change",v),()=>{t.removeEventListener("change",b),o.removeEventListener("click",w),c.removeEventListener("click",S),f.removeEventListener("change",v)}}function b(r){const d=r.target.files;d.length!==0&&(x(),P(d[0]).then(a=>{i=a,m(h),L(a),o.disabled=!1,c.disabled=!1}))}function v(r){s.setStrokeColor(O[r.target.value])}function g(r){r.classList.remove("hidden")}function m(r){r.classList.add("hidden")}function L(r){e=document.createElement("canvas"),n=document.createElement("canvas"),n.classList.add("absolute"),n.classList.add("top-0"),e.height=r.height,e.width=r.width,n.height=r.height,n.width=r.width,e.getContext("2d").drawImage(r,0,0),s=I(n),u.appendChild(e),u.appendChild(n)}function w(){s&&s.clear()}function x(){e&&e.remove(),s&&(s.dispose(),s=null),m(l),g(u)}function q(){o.disabled=!0,c.disabled=!0,E()}function C(r,d,a){l.querySelector("#output-original").src=r.src,l.querySelector("#output-segmented").src=d.replace(/'/g,""),l.querySelector("#output-dimension").innerText=`${e.height} x ${e.width}`,l.querySelector("#output-processing-time").innerText=`${a} s`}function S(){if(!window.google)return;let r=e.toDataURL(),d=n.toDataURL();g(y);const a=new Date().getTime();window.google.colab.kernel.invokeFunction("notebook.segment",[r,d],{}).then(p=>{const k=new Date().getTime();C(i,p.data["text/plain"],(k-a)/1e3),m(u),g(l)}).catch(p=>{console.log("whoops something went wrong"),console.log(p)}).finally(()=>{m(y)})}return{run:q,renderImage:L}}const T=R();T.run();
    </script>
  </body>
</html>
'''))